In [1]:
cd F:\MTP

F:\MTP


In [2]:
import os
import torch
from collections import defaultdict
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf

In [3]:
import os
path = os.listdir('EmoDB/wav')

We extract speaker embeddings(x-vectors) with a pretrained TDNN model using SpeechBrain from each utternace with a size of 512.

In [4]:
import torchaudio
from speechbrain.pretrained import EncoderClassifier
classifier = EncoderClassifier.from_hparams(source="speechbrain/spkrec-xvect-voxceleb", savedir="pretrained_models/spkrec-xvect-voxceleb")

Each utterance from the database is name according to the scheme :
    Positions 1-2: number of speaker
    Positions 3-5: code for text
    Position 6: emotion (sorry, letter stands for german emotion word)
    Position 7: if there are more than two versions these are numbered a, b, c....

Example: 03a01Fa.wav is the audio file from Speaker 03 speaking text a01 with
the emotion "Freude" (Happiness in German).

So now,we extract the emotion from the filenames and consider it as y in our model.

In [5]:
emotion_code = {
    'W':0, #anger
    'L':1, #boredom
    'E':2, #disgust
    'A':3, #fear
    'F':4, #happy
    'T':5, #sad
    'N':6  #neutral
}

speaker_code = {
    '03':0,
    '08':1,
    '09':2,
    '10':3,
    '11':4,
    '12':5,
    '13':6,
    '14':7,
    '15':8,
    '16':9
}

In [6]:
x_vec_size = 512
X = torch.zeros([len(path),x_vec_size])
y = torch.zeros([len(path),7],dtype = int)
for i in range(len(path)):
    signal, fs =torchaudio.load('EmoDB/wav/'+path[i])
    embeddings = classifier.encode_batch(signal)
    X[i] = embeddings[0][0]
    y[i][emotion_code[path[i][5]]]=1

In [7]:
# ys = torch.zeros([len(path),10],dtype = int)
# yg = torch.zeros([len(path),2],dtype = int)
# for i in range(len(path)):
#     ys[i][speaker_code[path[i][:2]]]=1
#     yg[i][gender_code[path[i][:2]]]=1

In [8]:
#print(X.shape,y.shape,ys.shape)

In [9]:
#print(ys[0])

Convert them into numpy arrays so that we can train them easily

In [10]:
X=X.detach().numpy()
y=y.detach().numpy()
#ys=ys.detach().numpy()

In [11]:
dict_X=defaultdict(list)
dict_y=defaultdict(list)
#dict_ys=defaultdict(list)
for i in range(len(path)):
    dict_X[path[i][:2]].append(X[i])
    dict_y[path[i][:2]].append(y[i])
   # dict_ys[path[i][:2]].append(ys[i])

In [12]:
#print(X.shape,ys.shape)

To make this evaluation speaker independent, we separate the data speaker-wise and use leave-one-out validation.

In [13]:
dict = {0:"03",1:"08",2:"09",3:"10",4:"11",5:"12",6:"13",7:"14",8:"15",9:"16"}

In [14]:
gender_code = {
    '03':0,
    '08':1,
    '09':1,
    '10':0,
    '11':0,
    '12':0,
    '13':1,
    '14':1,
    '15':0,
    '16':1
}

In [15]:
#requirements


from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import Adam
from keras.utils import np_utils
from sklearn import metrics
from sklearn.metrics import accuracy_score
from keras.callbacks import ModelCheckpoint


Since the x-vectors are extracted before the last layer from a pretrained model, we just add softmax layer to get the output layer.

In [16]:
def train_and_test(X,y,X_test,y_test,X_valid,y_valid):
    model = Sequential()
    model.add(Dense(7, input_shape=(len(X[0]),)))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
    checkpoint_filepath = './tmp/checkpoint'
    model_checkpoint_callback =ModelCheckpoint(filepath=checkpoint_filepath,monitor='val_accuracy',mode='max',save_best_only=True)
    model.fit(X,y,batch_size=20, epochs=200,callbacks=model_checkpoint_callback,validation_data=(X_valid,y_valid))
    loss,acc=model.evaluate(X_test,y_test)
    print(acc)
    return acc

In [17]:
def train_and_test_speak(X,y,X_test,y_test,X_valid,y_valid):
    model = Sequential()
    model.add(Dense(10, input_shape=(len(X[0]),)))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
    checkpoint_filepath = './tmp/checkpoint_speakers'
    model_checkpoint_callback =ModelCheckpoint(filepath=checkpoint_filepath,monitor='val_accuracy',mode='max',save_best_only=True)
    model.fit(X,y,batch_size=20, epochs=200,callbacks=model_checkpoint_callback,validation_data=(X_valid,y_valid))
    loss,acc=model.evaluate(X_test,y_test)
    print(acc)
    return acc

In [ ]:
temp = 9
acc = 0
accuracies = [0]*10
for i in range(10):
    if temp == 0:
        valid = 9
    else:
        valid = temp-1
    
    a = list((dict_X[dict[i]] for i in range(10) if i not in [temp,valid] ))
    b = list((dict_y[dict[i]] for i in range(10) if i not in [temp,valid]))
    X_ = np.concatenate(a)
    X_test = np.array(dict_X[dict[temp]])
    X_valid = np.array(dict_X[dict[valid]])
    y_test = np.array(dict_y[dict[temp]])
    y_valid = np.array(dict_y[dict[valid]])
    y_ = np.concatenate(b)
    temp -= 1
    print(9-temp)
    temp_acc = train_and_test(X_,y_,X_test,y_test,X_valid,y_valid)
    accuracies[temp]=temp_acc
    print(f'Accuracy for iteration: {temp_acc}')
    acc += temp_acc

leave-one-speaker-out validation accuracies

In [19]:
accuracies

[0.568965494632721,
 0.6744186282157898,
 0.7631579041481018,
 0.6545454263687134,
 0.7428571581840515,
 0.688524603843689,
 0.6666666865348816,
 0.6428571343421936,
 0.8873239159584045,
 0.8775510191917419]

In [20]:
print("Overall accuracy :"+ str(acc*10)+"%" )

Overall accuracy :71.66867971420288%


In [21]:
#model for speaker recognition
# Xtrain1, Xtest, ytrain1, ytest = train_test_split(X.numpy(), ys.numpy(), test_size=0.3, random_state=42)
# Xtrain, Xvalid, ytrain, yvalid = train_test_split(Xtrain1, ytrain1, test_size=0.15, random_state=42)
# model = Sequential()
# model.add(Dense(10, input_shape=(len(X[0]),)))
# model.add(Activation('softmax'))
# model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
# checkpoint_filepath = './tmp/checkpoint_speakers'
# model_checkpoint_callback =ModelCheckpoint(filepath=checkpoint_filepath,monitor='val_accuracy',mode='max',save_best_only=True)
# model.fit(Xtrain,ytrain,batch_size=20, epochs=200,callbacks=model_checkpoint_callback,validation_data=(Xvalid,yvalid))
# loss,acc=model.evaluate(Xtest,ytest)
# print(acc)

In [22]:
# model for gender recognition
# Xtrain1, Xtest, ytrain1, ytest = train_test_split(X.numpy(), yg.numpy(), test_size=0.3, random_state=42)
# Xtrain, Xvalid, ytrain, yvalid = train_test_split(Xtrain1, ytrain1, test_size=0.15, random_state=42)
# model = Sequential()
# model.add(Dense(2, input_shape=(len(X[0]),)))
# model.add(Activation('softmax'))
# model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')
# checkpoint_filepath = './tmp/checkpoint_speakers'
# model_checkpoint_callback =ModelCheckpoint(filepath=checkpoint_filepath,monitor='val_accuracy',mode='max',save_best_only=True)
# model.fit(Xtrain,ytrain,batch_size=20, epochs=200,callbacks=model_checkpoint_callback,validation_data=(Xvalid,yvalid))
# loss,acc=model.evaluate(Xtest,ytest)
# print(acc)


In [23]:

M31_X,M31_y=np.array(dict_X["03"]),np.array(dict_y["03"])
F34_X,F34_y=np.array(dict_X["08"]),np.array(dict_y["08"])
F21_X,F21_y=np.array(dict_X["09"]),np.array(dict_y["09"])
M32_X,M32_y=np.array(dict_X["10"]),np.array(dict_y["10"])
M26_X,M26_y=np.array(dict_X["11"]),np.array(dict_y["11"])
M30_X,M30_y=np.array(dict_X["12"]),np.array(dict_y["12"])
F32_X,F32_y=np.array(dict_X["13"]),np.array(dict_y["13"])
F35_X,F35_y=np.array(dict_X["14"]),np.array(dict_y["14"])
M25_X,M25_y=np.array(dict_X["15"]),np.array(dict_y["15"])
F31_X,F31_y=np.array(dict_X["16"]),np.array(dict_y["16"])

In [24]:
import librosa

In [25]:
melspecs =[0 for i in range(len(path))]
Y = torch.zeros([len(path),7],dtype = int)
for i in range(len(path)):
    r, sr = librosa.load('EmoDB/wav/'+path[i], res_type='kaiser_fast')
    melspec = librosa.feature.melspectrogram(y=r, sr=sr,n_fft = 512, hop_length=160, win_length=320,n_mels=24)
    melspecs[i]=melspec
    Y[i][emotion_code[path[i][5]]]=1

In [26]:
testfeats_X=defaultdict(list)
testfeats_y=defaultdict(list)
for i in range(len(path)):
    testfeats_X[path[i][:2]].append(melspecs[i])
    testfeats_y[path[i][:2]].append(Y[i])

In [27]:
trainfeats_X=defaultdict(list)
trainfeats_y=defaultdict(list)
for j in range(len(path)):    
    for i in range(0,len(melspecs[j][1]),50):
        temp = melspecs[0][:,i:i+100]
        if len(temp[1])==100:
            trainfeats_X[path[j][:2]].append(melspecs[0][:,i:i+100])
            trainfeats_y[path[j][:2]].append(Y[j])

In [28]:
train_set=[]
for j in range(10):    
    for i in range(0,len(trainfeats_X[dict[j]])):
        train_set.append((torch.tensor(trainfeats_X[dict[j]][i]),trainfeats_y[dict[j]][i]))

In [29]:
import random
random.shuffle(train_set)

In [30]:
def reshapex(x):  
    return torch.reshape(x,(1,24,x.shape[1]))
def reshapey(x):  
    return torch.reshape(x,(1,7))

In [31]:
import torch  
import torch.nn as nn
# import torchvision.datasets as datasets
# import torchvision.transforms as transforms
import speechbrain as sb
from speechbrain.nnet.pooling import StatisticsPooling
from speechbrain.nnet.CNN import Conv1d
from speechbrain.nnet.linear import Linear
from speechbrain.nnet.activations import Softmax
from speechbrain.nnet.normalization import BatchNorm1d

class Xvector(torch.nn.Module):
    """This model extracts X-vectors for speaker recognition and diarization.
    Arguments
    ---------
    device : str
        Device used e.g. "cpu" or "cuda".
    activation : torch class
        A class for constructing the activation layers.
    tdnn_blocks : int
        Number of time-delay neural (TDNN) layers.
    tdnn_channels : list of ints
        Output channels for TDNN layer.
    tdnn_kernel_sizes : list of ints
        List of kernel sizes for each TDNN layer.
    tdnn_dilations : list of ints
        List of dilations for kernels in each TDNN layer.
    lin_neurons : int
        Number of neurons in linear layers.
    Example
    -------
    >>> compute_xvect = Xvector('cpu')
    >>> input_feats = torch.rand([5, 10, 40])
    >>> outputs = compute_xvect(input_feats)
    >>> outputs.shape
    torch.Size([5, 1, 512])
    """
    
    def __init__(
        self,
        device,
        activation,
        tdnn_blocks,
        tdnn_channels,
        tdnn_kernel_sizes,
        tdnn_dilations,
        lin_neurons,
        in_channels,
    ):
        
        super().__init__()
        self.blocks = nn.ModuleList()

        # TDNN layers
        for block_index in range(tdnn_blocks):
            out_channels = tdnn_channels[block_index]
            self.blocks.extend(
                [
                    Conv1d(
                        in_channels=in_channels,
                        out_channels=out_channels,
                        kernel_size=tdnn_kernel_sizes[block_index],
                        dilation=tdnn_dilations[block_index],
                    ),
                    activation(),
                    BatchNorm1d(input_size=out_channels),
                ]
            )
            in_channels = tdnn_channels[block_index]

        # Statistical pooling
        self.blocks.append(StatisticsPooling())

        # Final linear transformation
        self.blocks.append(
            Linear(
                input_size=out_channels * 2,
                n_neurons=lin_neurons,
                bias=True,
                combine_dims=False,
            )
        )
        self.blocks.append(Linear(7,input_size=lin_neurons))
#         self.blocks.append(Softmax())
        
        

    def forward(self, x, lens=None):
        """Returns the x-vectors.
        Arguments
        ---------
        x : torch.Tensor
        """

        for layer in self.blocks:
            try:
                x = layer(x, lengths=lens)
            except TypeError:
                x = layer(x)
        return x


In [32]:
compute_xvect = Xvector("cpu",torch.nn.LeakyReLU,5,[512, 512, 512, 512, 1500],[5, 3, 3, 1, 1],[1, 2, 3, 1, 1],512,100)

In [33]:
import torch.optim as optim
optimizer = optim.Adam(compute_xvect.parameters(), lr = 0.0001)
criterion = nn.CrossEntropyLoss()
epochs = 5

for epoch in range(epochs):
    
    for data in train_set[:1840]:    
        
        X, y = data
        compute_xvect.zero_grad()
        output = compute_xvect(reshapex(X))
        y=reshapey(y)
        loss=criterion(output[0], torch.argmax(y).view(-1))
        loss.backward()
        optimizer.step()
        

In [34]:
corr = 0
total = 0
with torch.no_grad():
    for data in train_set[1840:2140]:
        X,y=data
        output = compute_xvect(reshapex(X))
        for idx,i in enumerate(output):
            if torch.argmax(i)==y[idx]:
                corr+=1
            total+=1
            
print("Accuracy = "+str(corr*100/total))

Accuracy = 75.33333333333333
